# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [2]:
!pip install openai python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [3]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [7]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('IronAPI')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [8]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [9]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 World Championship in 2010. He was driving for the Red Bull Racing team.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [10]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [11]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 championship was won by Fernando Alonso, driving for Renault.


In [12]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 Championship was won by Lewis Hamilton driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [13]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [15]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [16]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [17]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [19]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in stock markets.\n\n'},
    {'role':'user', 'content':'What was the best performing stock in 2023?'},
    {'role':'assistant', 'content':"""Stock: Coinbase. \nFounded: 2012. \nCEO: Brian Armstrong. """},
    {'role':'user', 'content':'What was the best performing stock in 2022?'},
    {'role':'assistant', 'content':"""Stock: Occidental Petroleum. \nFounded: 1920. \nCEO: Vicki Hollub. """},
]

print(return_OAIResponse("What is the best performing stock in 2024?", context_user))

I'm unable to provide real-time updates on specific stock performance, as it can fluctuate frequently. I recommend checking financial news sources or stock market websites for the latest information on the best-performing stock in 2024.


In [20]:
print(return_OAIResponse("What was the best performing stock in 2021?", context_user))

Stock: Moderna Inc. 
Founded: 2010. 
CEO: Stéphane Bancel.


In [22]:
context_user = [
    {'role':'system', 'content':
     """You are an expert historian.

     When was slavery abolished in Russia?
     Year: 1723.
     Decision made by: Peter the Great.

     When was slavery abolished in Germany?
     Year: 1837.
     Decision made by: German Parliament."""}
]
print(return_OAIResponse("When was slavery abolished in the US?", context_user))

Slavery was officially abolished in the United States with the ratification of the 13th Amendment to the Constitution on December 6, 1865, following the end of the American Civil War.


In [23]:
context_user = [
    {'role':'system', 'content':
     """You are an expert historian. Provide your answer in the following format

     When was slavery abolished in Russia?
     Year: 1723.
     Decision made by: Peter the Great.

     When was slavery abolished in Germany?
     Year: 1837.
     Decision made by: German Parliament."""}
]
print(return_OAIResponse("When was slavery abolished in the US?", context_user))

When was slavery abolished in the US?
Year: 1865. 
Decision made by: The 13th Amendment to the United States Constitution, which was passed by Congress on January 31, 1865, and ratified on December 6, 1865.


In [28]:
context_user = [
    {'role':'system', 'content':
     """You are an expert historian. Provide your answer in the following format, stipulate only two pieces of information the year and the decision-making body without any further information

     When was slavery abolished in Russia?
     Year: 1723.
     Decision-making body: Tsar Peter the Great.

     When was slavery abolished in Germany?
     Year: 1837.
     Decision-making body: German Parliament."""}
]
print(return_OAIResponse("When was slavery abolished in the US?", context_user))

Year: 1865.  
Decision-making body: US Congress.


In [29]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in non-cooperative game theory.\n\n'},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the prisoners dilemma game?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Confess, Confess)."""},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the stag hunt game?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Stag, Stag), (Hare, Hare). """},
]

print(return_OAIResponse("What are pure strategy Nash equilibria in the chicken game?", context_user))

In the chicken game, the pure strategy Nash equilibria are (Swerve, Straight) and (Straight, Swerve).


In [31]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in non-cooperative game theory.\n\n'},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the prisoners dilemma game?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Confess, Confess)."""},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the stag hunt game?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Stag, Stag), (Hare, Hare). """},
]

print(return_OAIResponse("What are pure strategy Nash equilibria in this game: 	Raise Limit	Keep Limit	Dodge, Raise Limit	(45,55)(50,50)(40,60), Keep Limit (60,40)(55,45)(50,50), Dodge (45,55)(55,45)(40,60)?", context_user))

The pure strategy Nash equilibria in this game are: (Raise Limit, Keep Limit), (Keep Limit, Raise Limit), and (Keep Limit, Keep Limit).


In [32]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in non-cooperative game theory.\n\n'},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the prisoners dilemma game: Confess Dont confess, Confess (-6,-6) (0, -10), Dont confess (-10,0) (-1,-1)?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Confess, Confess)."""},
    {'role':'user', 'content':'What are pure strategy Nash equilibria in the stag hunt game: Stag Hare, Stag (3,3) (0, 1), Hare (1,0) (1,1)?'},
    {'role':'assistant', 'content':"""List of Nash equilibria: (Stag, Stag), (Hare, Hare). """},
]

print(return_OAIResponse("What are pure strategy Nash equilibria in this game: 	Raise Limit	Keep Limit	Dodge, Raise Limit	(45,55)(50,50)(40,60), Keep Limit (60,40)(55,45)(50,50), Dodge (45,55)(55,45)(40,60)?", context_user))

The pure strategy Nash equilibria in this game are: (Raise Limit, Keep Limit) and (Keep Limit, Dodge).
